In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import MinMaxScaler,StandardScaler,LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, r2_score
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout
from xgboost import XGBClassifier

In [7]:
data=pd.read_csv('email_spam.csv')

In [11]:
data.sample(6)

,title,text,type
61,Time is of the Essence!,"Dear Customer,\n\nThe 2022 filing season is al...",spam
29,Gut Health & Weight Loss Plan - Role of Supple...,"Hi Maryam,,\n\nPlease find the information on ...",not spam
2,?? Your GitHub launch code,"Here's your GitHub launch code, @Mortyj420!\n ...",not spam
20,Do you have $7 Walid?,"Hello Walid,\n\nDo you have $7?\n\nI ask becau...",spam
58,Telemarketing + 19 new jobs - Job Alert from J...,"Jobstreet.com\n\nHi ANGGRAENI, we've got new j...",not spam
54,"Sathya!, welcome to Snapchat+!","As a Snapchat+ subscriber, you now have access...",not spam


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from xgboost import XGBClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [109]:
X = data['text'] 
le = LabelEncoder()
y = le.fit_transform(data['type']) 

vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')

In [117]:
def remove_non_dictionary_words(text):
    words = re.sub(r'[^a-zA-Z\s]', '', text).split()
    clean_words = [word for word in words if word.lower() in valid_words]
    return ' '.join(clean_words)

In [111]:
X_vectorized = vectorizer.fit_transform(X)
X_df = pd.DataFrame(X_vectorized.toarray(), columns=vectorizer.get_feature_names_out())

In [113]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y, test_size=0.2, random_state=42)

## XGBoost

In [64]:
xgb_model = XGBClassifier()

In [66]:
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [68]:
y_pred = xgb_model.predict(X_test)

In [70]:
accuracy_score(y_test, y_pred)

0.6470588235294118

In [72]:
print(classification_report(y_test, xgb_preds))

              precision    recall  f1-score   support

           0       0.69      0.82      0.75        11
           1       0.50      0.33      0.40         6

    accuracy                           0.65        17
   macro avg       0.60      0.58      0.57        17
weighted avg       0.62      0.65      0.63        17



### Neural Networks

In [41]:
from tensorflow.keras.optimizers import Adam

nn_model = Sequential([
    Dense(512, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid') 
])


C:\Users\dimit\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [99]:
nn_model.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss='binary_crossentropy',
    metrics=['accuracy'])

history = nn_model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 171ms/step - accuracy: 0.9311 - loss: 0.0644 - val_accuracy: 0.7059 - val_loss: 0.6783
Epoch 2/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.9620 - loss: 0.0431 - val_accuracy: 0.7059 - val_loss: 0.6572
Epoch 3/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.9581 - loss: 0.0469 - val_accuracy: 0.7059 - val_loss: 0.6240
Epoch 4/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 1.0000 - loss: 0.0353 - val_accuracy: 0.7059 - val_loss: 0.5980
Epoch 5/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.9773 - loss: 0.0398 - val_accuracy: 0.7059 - val_loss: 0.5794
Epoch 6/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.9616 - loss: 0.0373 - val_accuracy: 0.7059 - val_loss: 0.5752
Epoch 7/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.9808 - loss: 0.0341 - val_accuracy: 0.7059 - val_loss: 0.5709
Epoch 8/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.9503 - loss: 0.0482 - val_accuracy: 0.7059 - val_loss: 0.5510

In [101]:
y_pred = (nn_model.predict(X_test) > 0.5).astype("int32")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step


In [103]:
accuracy_score(y_test, y_pred)

0.7058823529411765

In [105]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.69      1.00      0.81        11
           1       1.00      0.17      0.29         6

    accuracy                           0.71        17
   macro avg       0.84      0.58      0.55        17
weighted avg       0.80      0.71      0.63        17

